# Import Libraries

In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle

# Model

In [2]:
class Chatbot:
    def __init__(self, vAR_data):
        self.vAR_data = vAR_data #input json data
        self.vAR_patterns = [] #patterns in input json data
        self.vAR_tags = [] #tags in input json data
        self.vAR_labels = [] #labels in input json data (distinct tags)
        self.vAR_responses = [] #responses in input json data
        self.vAR_num_classes = 0 #total number of classes(tags)
        self.vAR_max_word_size = 1000 #tokenizer keeps the most common 1000 words
        self.vAR_embedding_dim = 16 #output dimension of embedding layer
        self.vAR_max_len = 20 #max length for pad sequence  which is input of embedding layer
        self.vAR_oov_token = "<OOV>" #text to replace when out of vocabulary occurs
        self.vAR_encoder = None #encoder
        self.vAR_tokenizer = None #tokenizer
        self.vAR_padded_sequence = None #padded sequence
        self.vAR_model = None #model
        self.vAR_epochs = 500 #epochs
        
    def extract_data(self):
        #extract tags, patterns, labels from json data
        
        for intent in self.vAR_data['intents']:# loop through input file
            for p in intent['patterns']:#for each elements in patterns
                self.vAR_patterns.append(p)#append each elements in patterns
            
            #append tags n times, where n is number of elements in pattern
            self.vAR_tags.extend([intent['tag']]*len(intent['patterns']))

            self.vAR_responses.append(intent['responses']) #append list of responses
            self.vAR_labels.append(intent['tag']) #append distinct tags
            self.vAR_num_classes = len(self.vAR_labels) #contains number of distinct tags
            
    def encode(self):
        #label encoder
        self.vAR_encoder = LabelEncoder()
        self.vAR_tags = self.vAR_encoder.fit_transform(self.vAR_tags) #encoding each tags
        
    def tokenize(self):
        # Tokenizer- removes punctuation, tokenized
        self.vAR_tokenizer = Tokenizer(num_words=self.vAR_max_word_size, oov_token=self.vAR_oov_token) #tokenizer object
                
        self.vAR_tokenizer.fit_on_texts(self.vAR_patterns) # each word -> unique index (low index - more frequent)
        print('Word Index :', self.vAR_tokenizer.word_index, end='\n')
        
        #convert words in pattern and convert into integer based on self.vAR_tokenizer.word_index
        vAR_sequences = self.vAR_tokenizer.texts_to_sequences(self.vAR_patterns)
        
        #convert integers to 2d array with padding 0.
        #if length exceeds 20 then truncate later elements
        self.vAR_padded_sequences = pad_sequences(vAR_sequences, truncating='post', maxlen=self.vAR_max_len)
        
    def create_model(self):
        self.vAR_model = Sequential()
        #Embedding layer - convert each word into a fixed length vector of defined size with real values
        #input_dim : vocabulay size (here 1000)
        #output_dim : each word is represented as a vector of size 16 (here)
        #input_length : length of input padded sequence for this layer is 20
        self.vAR_model.add(Embedding(input_dim=self.vAR_max_word_size, output_dim=self.vAR_embedding_dim, input_length=self.vAR_max_len))
        self.vAR_model.add(GlobalAveragePooling1D())
        self.vAR_model.add(Dense(16, activation='relu'))
        self.vAR_model.add(Dense(16, activation='relu'))
        self.vAR_model.add(Dense(self.vAR_num_classes, activation='softmax'))

        #probability of each class
        self.vAR_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        self.vAR_model.summary()
        
    def train_model(self):
        self.vAR_model.fit(self.vAR_padded_sequences, np.array(self.vAR_tags), epochs=self.vAR_epochs)
        
    def save_objects(self):
        # saving tokenizer
        with open('../Utility/tokenizer.pickle', 'wb') as f:
            pickle.dump(self.vAR_tokenizer, f)#, protocol=pickle.HIGHEST_PROTOCOL)

        # saving label encoder
        with open('../Utility/label_encoder.pickle', 'wb') as f:
            pickle.dump(self.vAR_encoder, f)#, protocol=pickle.HIGHEST_PROTOCOL)
            
        self.vAR_model.save("../Utility/chat_model.h5")
        
    def RUN_ALL(self):
        self.extract_data()
        self.encode()
        self.tokenize()
        self.create_model()
        self.train_model()
        self.save_objects()

# Import Data

In [3]:
with open('../Utility/DSAI_Intents.json') as file:
    vAR_data = json.load(file)

# Create Object & Run All

In [4]:
chat = Chatbot(vAR_data)
chat.RUN_ALL()

Word Index : {'<OOV>': 1, 'you': 2, 'can': 3, 'i': 4, 'me': 5, 'the': 6, 'how': 7, 'your': 8, 'are': 9, 'what': 10, 'a': 11, 'courses': 12, 'be': 13, 'help': 14, "deepsphere's": 15, 'get': 16, 'for': 17, 'name': 18, 'do': 19, 'send': 20, 'office': 21, 'contact': 22, 'please': 23, 'my': 24, 'done': 25, 'in': 26, "what's": 27, 'see': 28, 'later': 29, 'thanks': 30, 'appreciate': 31, 'it': 32, 'of': 33, 'doing': 34, 'good': 35, 'is': 36, 'have': 37, 'service': 38, 'website': 39, 'number': 40, 'newsletter': 41, 'need': 42, 'support': 43, 'textbooks': 44, 'where': 45, 'order': 46, 'will': 47, 'scheduled': 48, 'only': 49, 'hi': 50, 'hey': 51, 'up': 52, 'there': 53, 'hello': 54, 'hay': 55, 'bye': 56, 'goodbye': 57, 'catch': 58, 'thank': 59, 'so': 60, 'kind': 61, 'really': 62, 'hope': 63, 'whats': 64, 'who': 65, 'call': 66, 'list': 67, "organization's": 68, 'deepsphere': 69, 'link': 70, "link'": 71, 'reach': 72, 'staff': 73, 'signup': 74, 'to': 75, 'guys': 76, 'achievements': 77, 'accomplishmen

2/2 [==============================] - 0s 3ms/step - loss: 2.5386 - accuracy: 0.0893
Epoch 61/500
2/2 [==============================] - 0s 3ms/step - loss: 2.5335 - accuracy: 0.0893
Epoch 62/500
2/2 [==============================] - 0s 4ms/step - loss: 2.5273 - accuracy: 0.0893
Epoch 63/500
2/2 [==============================] - 0s 4ms/step - loss: 2.5213 - accuracy: 0.0893
Epoch 64/500
2/2 [==============================] - 0s 6ms/step - loss: 2.5152 - accuracy: 0.0893
Epoch 65/500
2/2 [==============================] - 0s 4ms/step - loss: 2.5093 - accuracy: 0.0893
Epoch 66/500
2/2 [==============================] - 0s 5ms/step - loss: 2.5034 - accuracy: 0.0893
Epoch 67/500
2/2 [==============================] - 0s 5ms/step - loss: 2.4979 - accuracy: 0.1250
Epoch 68/500
2/2 [==============================] - 0s 4ms/step - loss: 2.4919 - accuracy: 0.1607
Epoch 69/500
2/2 [==============================] - 0s 3ms/step - loss: 2.4861 - accuracy: 0.1429
Epoch 70/500
2/2 [===============

2/2 [==============================] - 0s 3ms/step - loss: 0.9692 - accuracy: 0.8036
Epoch 226/500
2/2 [==============================] - 0s 3ms/step - loss: 0.9597 - accuracy: 0.8036
Epoch 227/500
2/2 [==============================] - 0s 4ms/step - loss: 0.9516 - accuracy: 0.8036
Epoch 228/500
2/2 [==============================] - 0s 5ms/step - loss: 0.9410 - accuracy: 0.8036
Epoch 229/500
2/2 [==============================] - 0s 5ms/step - loss: 0.9314 - accuracy: 0.8214
Epoch 230/500
2/2 [==============================] - 0s 4ms/step - loss: 0.9222 - accuracy: 0.8214
Epoch 231/500
2/2 [==============================] - 0s 4ms/step - loss: 0.9123 - accuracy: 0.8214
Epoch 232/500
2/2 [==============================] - 0s 5ms/step - loss: 0.9030 - accuracy: 0.8214
Epoch 233/500
2/2 [==============================] - 0s 5ms/step - loss: 0.8939 - accuracy: 0.8214
Epoch 234/500
2/2 [==============================] - 0s 5ms/step - loss: 0.8849 - accuracy: 0.8214
Epoch 235/500
2/2 [=====

2/2 [==============================] - 0s 4ms/step - loss: 0.1380 - accuracy: 1.0000
Epoch 390/500
2/2 [==============================] - 0s 3ms/step - loss: 0.1367 - accuracy: 1.0000
Epoch 391/500
2/2 [==============================] - 0s 4ms/step - loss: 0.1348 - accuracy: 1.0000
Epoch 392/500
2/2 [==============================] - 0s 4ms/step - loss: 0.1333 - accuracy: 1.0000
Epoch 393/500
2/2 [==============================] - 0s 4ms/step - loss: 0.1319 - accuracy: 1.0000
Epoch 394/500
2/2 [==============================] - 0s 6ms/step - loss: 0.1306 - accuracy: 1.0000
Epoch 395/500
2/2 [==============================] - 0s 4ms/step - loss: 0.1289 - accuracy: 1.0000
Epoch 396/500
2/2 [==============================] - 0s 4ms/step - loss: 0.1274 - accuracy: 1.0000
Epoch 397/500
2/2 [==============================] - 0s 5ms/step - loss: 0.1261 - accuracy: 1.0000
Epoch 398/500
2/2 [==============================] - 0s 4ms/step - loss: 0.1246 - accuracy: 1.0000
Epoch 399/500
2/2 [=====

In [5]:
asd

NameError: name 'asd' is not defined

In [ ]:
#save tokenizer and encoder
def chat():
    max_len = 20
    
    while True:
        print("User: ",end="")
        inp = input()
        if inp.lower() == "quit":
            break

        t = tokenizer.texts_to_sequences([inp])
        p = pad_sequences(t, truncating='post', maxlen=max_len)
        result = model.predict(p)
        pred_class = np.argmax(result)
        tag = encoder.inverse_transform([pred_class])

        for i in data['intents']:
            if i['tag'] == tag:
                print(np.random.choice(i['responses']))

chat()